# Notes

This notebook was originally created and stored on Google Colaboratory. For convenience, output has been saved according to the intended input (stored on Google Drive). This is for display and not intended to be ran locally.

# K-Means and WNMF Test Driver
This notebook contains methods for testing various parameters of our recommender system pipeline.
Our current hypothetical pipeline is:
1. Review data will be clustered on the user side
2. The clusters will be set to the server
3. The server will append the clusters to the main matrix.
4. The server will perform WNMF on the clusters
5. The U and V matrices will be sent out to the users
6. The users will have the U and V matrices multiplied to receive recommendations

The full test here will mainly focus on ensuring the recommendations when updating with new clusters still provide high quality predictions (low in error). The test pipeline will be:
1.   Take user reviews chronologically, drop the earliest reviews up to a point, and use a subset of 1000 as our utility matrix
2.   Cluster this utility matrix with K-Means clustering on the users
3. Perform WNMf on the clustered matrix and get the U and V matrices
4. For the given U and V matrices, find users not within the clusters but have businesses in their review set that overlap so they can be tested on
5. Run the tests on the users
6. Perform steps 1 and 2 again but another set of reviews immediately after the previous set.
7. Append the new clusters to the old clusters
8. Perform steps 3 to 5 again.
9. Repeat process until at least half the reviews are used. 

Parameters we're going to test are:
1.   Latent factors when performing WNMF
2.   Number of clusters per K-Means round
3. Number of nearest neighbors when generating a recommendation

In [0]:
# Mounting drive
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [0]:
import math
import pandas as pd
import numpy as np
import scipy.stats
from sklearn.cluster import KMeans

In [0]:
# Import review data
fp = '/content/gdrive/My Drive/Recommender System Research Project 2019-2020/Individual Work Folder/Ian/yelp_detailed_small.csv'
df = pd.read_csv(fp)

In [0]:
# Prepare initial utility matrix without earliest ratings

# x is the chosen start year, where everything before is cut off
x = 2007                              

df = df.sort_values(by = 'date')
# For each year, exclude rows whose 'date' column contains the string of the year
for i in range(2004, x, 1):
  df = df[~df.date.str.contains(str(i))]
df = df.reset_index(drop = True)

## Server Side Processes
The "server" side processes include:
1. Storing and organizing clusters into a single UV matrix
2. Performing WNMF on the clusters
3. Storing the U and V matrix for the "users"

Other processes not included here but will be in the real program are: communicating U and V matrices to users. Receiving clusters from users. Mapping business ids

### How to use cluster generator

Create the GroupItemMatrix object which keeps track of which reviews have been clustered:

`g = GroupItemMatrix(df)`

Create clusters from next 1000 reviews:

`g.cluster_batch()`

Create k batches of clusters

`g.cluster_batches(k)`

Dataframe that contains all cluster batches created so far

`g.df`


In [0]:
# Cluster generator from reading data

class GroupItemMatrix:
  def __init__(self, source_df, batch_size=1000, clusters_per_batch=25):
    self.batch_size = batch_size
    self.clusters_per_batch = clusters_per_batch

    #keeps track of number of batches so far
    self.batch_count = 0

    #group-item matrix of cluster centroids we're creating
    self.df = pd.DataFrame()

    #source dataframe
    self.source_df = source_df

    #dataframe of used reviews: useful in cases where we need to know what reviews were used for the clusters
    self.old_df = pd.DataFrame(columns = source_df.columns)

  def cluster_batch(self):
    #select first batch of rows (1000)  ofsource df
    #curr_clusters is this batch of clusters
    self.curr_clusters = self.source_df.iloc[0:self.batch_size]
    self.source_df = self.source_df.iloc[self.batch_size:]

    #append the reviews being used to old df
    self.old_df = self.old_df.append(self.curr_clusters)

    #pivot data
    self.curr_clusters = self.curr_clusters.pivot(index = 'user_name_id', columns = 'business_name_id', values = 'stars')
    self.curr_clusters = self.curr_clusters.fillna(value = 0)

    #create KMeans clusters
    self.m = KMeans(n_clusters = self.clusters_per_batch)
    self.m.fit_predict(self.curr_clusters)

    #record column names to use later
    self.curr_columns = self.curr_clusters.columns

    #obtain clusters
    self.curr_clusters = pd.DataFrame(self.m.cluster_centers_)

    #put the column names back in
    self.curr_clusters.columns = self.curr_columns

    #make sure index is set to consecutively increase
    self.curr_clusters.index = [(i + (self.batch_count * self.clusters_per_batch)) for i in range(len(self.curr_clusters))]
    self.batch_count += 1

    #concatenate current batch of clusters with previous batches
    self.df = pd.concat([self.df, self.curr_clusters], axis=0, sort=False)
    self.df = self.df.fillna(value = 0)

  #if we want to run multiple batches at one time
  def cluster_batches(self, n):
    for i in range(n):
      self.cluster_batch()


In [0]:
# Server Side processes go here

#takes group-item matrix df and latent factors lf
def wnmf(df, lf):
  #choose limit of convergence
  limit = 0.2

  iterations = min(len(df), len(df.columns)) / lf
  #####################################################

  # Save columns
  columns = df.columns

  um = df.to_numpy()

  #convert NaN values to 0
  a = np.nan_to_num(um)

  #create matrix w of weights: 1 for observed values, else 0
  w = np.zeros((len(a), len(a[0])), dtype=int)
  for i in range(len(a)):
      for j in range(len(a[i])):
          if a[i][j] != 0:
              w[i][j] = 1

  #u/v matrix initialization with random values
  u = np.random.rand(len(a), lf)
  v = np.random.rand(lf, len(a[0]))

  iteration = 0
  prev_norm = 0
  curr_norm = 0
  change = 999999
  while(iteration < iterations and change > limit):
    u_it = np.nditer(u, flags=['multi_index'])
    v_it = np.nditer(v, flags=['multi_index'])
    while (not u_it.finished or not v_it.finished):
      if not u_it.finished:
        i, j = u_it.multi_index
        num = np.matmul((w[i,:] * a[i,:]), v.T[:,j])
        denom = np.matmul(w[i,:] * np.matmul(u[i,:], v), v.T[:,j])
        u[i][j] = u_it[0] * (num / denom)
        u_it.iternext()
      if not v_it.finished:
        i, j = v_it.multi_index
        num = np.matmul(u.T[i,:], (w[:,j] * a[:,j]))
        denom = np.matmul(u.T[i,:], (w[:,j] * np.matmul(u, v[:,j])))
        v[i][j] = v_it[0] * (num / denom)
        v_it.iternext()

    prev_norm = curr_norm
    curr_norm = np.linalg.norm((np.multiply(w, (a - np.matmul(u, v)))), ord='fro')
    change = abs(curr_norm - prev_norm)
    print('Iteration: ' + str(iteration) + ' Previous Norm: ' + str(prev_norm) + ' Current Norm: ' + str(curr_norm) + ' Change: ' + str(change))
    iteration += 1

  # Convert U and V back into a Dataframe with the proper information.
  u = pd.DataFrame(u)
  v = pd.DataFrame(v)
  v.columns = columns

  return u, v

## User Side Processes

The "user" side processes include:

1. For a particular user, take a subset of V where it only contains businesses the user rated (V2)
2. Starting with the 3rd business in chronological order, multiply each business before it with U and V, and find out the top K similar clusters. (If it is the 3rd business, use only 1 to 2. Next run we use the 4th, so use 1 to 3 and so on until we use the last business in the overlap set)
3. For that business, find the weighted recommendation from the top-K clusters.

For testing, we will
1. Record the error between the real rating and the predicted rating.
2. Do this for all businesses a user has and for all users that aren't in the cluster (but have businesses in the cluster)

After one cluster is done, we will expand the cluster to another 1000 ratings, and run the same tests again.

In [0]:
def find_candidate_reviews(g):
  df = g.source_df            # We take the dataframe of all reviews not used in the clusters
  exclude_users = g.old_df['user_name_id'].unique()   # We get the set of users are being used in the cluster, which we must exclude

  for i in exclude_users:
    df = df[df['user_name_id'] != i]

  # We find out how many years the canidate users have made reviews in
  user_dict = {}

  for i in range(len(df)):
    user_id = df.iloc[i, 0]
    rating_year = df.iloc[i, 3]
    rating_year = rating_year[0:4]
    
    if user_id not in user_dict:
      user_dict[user_id] = {rating_year}
    else:
      user_dict[user_id].add(rating_year)

  years_per_user = []
  for user, years in user_dict.items():
    years_per_user.append([user, len(years)])

  user_years = pd.DataFrame(years_per_user, columns=['user_name_id', 'years_with_rating'])

  # We sort for the top canidates
  user_years = user_years.sort_values(by='years_with_rating', ascending=False)
  user_years = user_years.reset_index()

  # We take the top 100 reviewers by year
  user_years = user_years['user_name_id'].iloc[0:100]

  # We find which reviews belong to the top 100 canidates so far and return that
  df = df[df['user_name_id'].isin(user_years)]
  return df

In [0]:
def build_user_test_set(candidate_reviews, g):
  # We reduce the reviews of canidate users down to the businesses that are in the clustered set 
  candidate_reviews = candidate_reviews[candidate_reviews['business_name_id'].isin(g.df.columns)]

  dict_ub = {}
  for i in candidate_reviews['user_name_id'].unique():
    df_temp = candidate_reviews[candidate_reviews['user_name_id'] == i]

    # We exclude anyone with less than 3 business reviews
    if(len(df_temp) >= 3):
      s = set(df_temp['business_name_id'])
      dict_ub[i] = s
  return dict_ub

In [0]:
def prediction_test(u, v, candidate_reviews, dict_ub, k):
  error = []
  main_v = v
  u = u.to_numpy()

  for i, j in dict_ub.items():
    # v is the subset of (latent factors x Businesses) where the businesses 
    # are both in the volunteer set and i's (the current test user's) review set.
    # Everytime we want to multiply by only the users businesses, we need to remake the v matrix
    v = main_v[[x for x in j]]
    v_columns = v.columns
    v = v.to_numpy()
    
    # matmul and convert back to DataFrame  
    uv = np.matmul(u, v)
    uv = pd.DataFrame(uv, columns = v_columns)

    # we reduce the review set down to i's reviews and sort by date
    r = candidate_reviews[candidate_reviews['user_name_id'] == i]
    r = r.sort_values('date')
    r = r[r.business_name_id.isin(j)]

    # We run tests from 2 to n - 1 user reviews (a user requires a minimum of 3 reviews). Each test will: run cosine similarity, find the top 3 clusters, and compare the user's review for x business.
    
    for x in range(2, len(r)):
      # Select the business to test
      currb = r.iloc[x, :]['business_name_id']

      # Generate the list of businesses to run cosine similarity
      testset = r.iloc[0:x, :]['business_name_id'].tolist()
      l1 = [r[r['business_name_id'] == business].iloc[0]['stars'] for business in testset]

      # For each cluster, we take the businesses currently being tested and run cosine similarity
      csd = []
      for cluster in range(len(uv)):
        l2 = [uv.iloc[cluster][business] for business in testset]
        csdistance = scipy.spatial.distance.cosine(l1, l2)
        csd.append((csdistance, cluster))
      csd.sort(reverse = True)
      csd = csd[0:k]
      # With the top-k clusters in csd, we generate a prediction on currb, multiplying the score from the cluster by its weight
      weights = [cluster[0] for cluster in csd]
      sw = sum(weights)
      weights = map(lambda weight: weight / sw, weights)
      prediction = 0
      for weight, cluster in zip(weights, csd):
        prediction += (uv.iloc[cluster[1]][currb] * weight)
      error.append(r.iloc[x, :]['stars'] - prediction)

  return error

In [0]:
def rsme(ls):
  ls = list(map(lambda x: x ** 2, ls))
  err = sum(ls) / len(ls)
  err = math.sqrt(err)
  return err

def test_parameters(lf, clusters, k):
  print("Using {} latent factors, {} clusters, and {} k neighbors".format(lf, clusters, k))
  g = GroupItemMatrix(df, clusters_per_batch = clusters)
  error = []
  for i in range(1):  
    g.cluster_batch()
    u, v = wnmf(g.df, lf)
    candidate_reviews = find_candidate_reviews(g)
    test_set = build_user_test_set(candidate_reviews, g)
    #error.append(prediction_test(u, v, candidate_reviews, test_set, k))
    error = prediction_test(u, v, candidate_reviews, test_set, k)
    print(rsme(error))
  # print(rsme(error))

test_parameters(10, 25, 3)

Using 10 latent factors, 25 clusters, and 3 k neighbors
Iteration: 0 Previous Norm: 0 Current Norm: 19.055333270297528 Change: 19.055333270297528
Iteration: 1 Previous Norm: 19.055333270297528 Current Norm: 14.354365371072017 Change: 4.70096789922551
Iteration: 2 Previous Norm: 14.354365371072017 Current Norm: 12.967051392873694 Change: 1.387313978198323
1.8143352071357106
Iteration: 0 Previous Norm: 0 Current Norm: 30.195652348723193 Change: 30.195652348723193
Iteration: 1 Previous Norm: 30.195652348723193 Current Norm: 24.038950905969084 Change: 6.156701442754109
Iteration: 2 Previous Norm: 24.038950905969084 Current Norm: 22.028996131690846 Change: 2.0099547742782384
Iteration: 3 Previous Norm: 22.028996131690846 Current Norm: 20.6311294208319 Change: 1.3978667108589455
Iteration: 4 Previous Norm: 20.6311294208319 Current Norm: 19.484398860149618 Change: 1.1467305606822826
3.4730510155271386
Iteration: 0 Previous Norm: 0 Current Norm: 36.44210596040676 Change: 36.44210596040676
Iter